In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read, get_array_diff

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Operacionalización de datos

### Alumno desertor
- Requisito: No se debe haber titulado para la carrera analizada

### Tipos de deserción

- Sale del sistema y no vuelve a ingresar
- Sale del sistema y vuelve después de 1, 2 ó 3 años y termina o sigue en el sistema (seudo deserción?):
    - Vuelve a la misma carrera en la misma institución (no deserta, "congela")
    - Vuelve a otra carrera en la misma institución
    - Vuelve a la misma carrera en otra institución
    - Vuelve a otra carrera en otra institución
- Sale del sistema, vuelve a ingresar, pero no termina (múltitple deserción?)

# Matriculados

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

- Se considera todos los matriculados en CFT / IP desde 2015

In [2]:
# Matriculados de CFT / IP
mat_cft_ip = pd.read_csv(f"{BASE}/tmp_data/fechas_titulacion.csv")
mat_cft_ip.head()

,codigo_unico,mrun,fecha_obtencion_titulo
0,I143S26C9J2V2,26379527.0,20220228.0
1,I260S22C108J1V1,4120917.0,20220228.0
2,I116S34C17J2V1,4348827.0,20220228.0
3,I260S21C263J2V1,20301532.0,20220228.0
4,I143S2C651J2V1,3498846.0,20220228.0


In [3]:
mat_cft_ip.isna().sum()

codigo_unico                    0
mrun                            0
fecha_obtencion_titulo    1194617
dtype: int64

In [4]:
# Se carga datos de matrícula para estos alumnos, considerando todas las instituciones
cols = [
    'cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 'dur_estudio_carr',
    'cod_inst', 'cod_carrera'
]

mruns = mat_cft_ip["mrun"].unique()

matriculados = pd.concat([
    iter_df_read(f, "mrun", mruns, sep=";", usecols=cols)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).drop_duplicates().reset_index(drop=True)

  0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
# Se agrega columna que indica titulación

if "fecha_obtencion_titulo" not in matriculados.columns:
    len_or = matriculados.shape[0]
    matriculados = pd.merge(
        matriculados,
        mat_cft_ip,
        how="left"
    )
    
    assert matriculados.shape[0] == len_or

## Pre procesamiento: Año de titulación

In [6]:
matriculados["anio_titulacion"] = matriculados["fecha_obtencion_titulo"].fillna("").astype(str).str[:4].replace([""], [np.nan]).astype(float)
matriculados["anio_titulacion"].describe().round()

count    1073280.0
mean        2019.0
std            1.0
min         2015.0
25%         2018.0
50%         2020.0
75%         2021.0
max         2022.0
Name: anio_titulacion, dtype: float64

## Pre procesamiento: Último año de matrícula por carrera

In [7]:
if "ultimo_anio_matricula" not in matriculados.columns:
    len_or = matriculados.shape[0]
    matriculados = pd.merge(
        matriculados,
        matriculados.groupby(["mrun", "codigo_unico"]).agg({
            "cat_periodo": "max"
        }).reset_index().rename(columns={
            "cat_periodo": "ultimo_anio_matricula"
        }),
        how="left"
    ).drop_duplicates().reset_index(drop=True)
    
    assert matriculados.shape[0] == len_or

In [17]:
matriculados["ultimo_anio_matricula"].describe().round()

count    3743349.0
mean        2020.0
std            2.0
min         2015.0
25%         2018.0
50%         2020.0
75%         2022.0
max         2022.0
Name: ultimo_anio_matricula, dtype: float64

## Pre prcesamiento: Deserciones

### Primer caso: Sale del sistema y no vuelve a ingresar
- Solo han tomado 1 carrera
- No se han titulado
- Puede o no tener "años en banco"
- No se han matriculado el año 2022

In [8]:
cft_ip = ["Institutos Profesionales", "Centros de Formación Técnica"]

In [9]:
ruts_matriculados_cft_ip = matriculados[
    matriculados["tipo_inst_1"].isin(cft_ip)
]["mrun"].unique()

print("Cant. personas que ingresaron a estudiar entre 2015 y 2021 y se matricularon en CFT / IP (universo total): ", ruts_matriculados_cft_ip.shape[0])

Cant. personas que ingresaron a estudiar entre 2015 y 2021 y se matricularon en CFT / IP (universo total):  1233136


In [10]:
cant_carreras = matriculados[
    matriculados["mrun"].isin(ruts_matriculados_cft_ip)
][["mrun", "codigo_unico"]].drop_duplicates().groupby(["mrun"]).size().reset_index(name="cant_carreras")

ruts_1_carrera = cant_carreras[cant_carreras["cant_carreras"] == 1]["mrun"].unique()

print("Dentro de los anteriores, cantidad que solo ha tomado 1 carrera: ", ruts_1_carrera.shape[0])

Dentro de los anteriores, cantidad que solo ha tomado 1 carrera:  741611


In [12]:
ruts_desertores_1 = matriculados[
    (matriculados["mrun"].isin(ruts_1_carrera))
    & (matriculados["anio_titulacion"].isna())
    & (matriculados["ultimo_anio_matricula"] < 2022)
]["mrun"].unique()

print("Dentro de los anteriores, no se han titulado ni matriculado el año 2022: ", ruts_desertores_1.shape[0])

Dentro de los anteriores, no se han titulado ni matriculado el año 2022:  349677


In [13]:
print(f"% Desertores Criterio 1: {np.round(ruts_desertores_1.shape[0] / ruts_matriculados_cft_ip.shape[0] * 100, 1)}%")

% Desertores Criterio 1: 28.4%


### Segundo caso:  Sale del sistema y vuelve a la misma carrera en la misma institución
- Dentro de una misma carrera, tiene años en blanco (1, 2 ó 3)
- En los años en blanco, no se ha matriculado en otra carrera
- Se titula o se matricula en el año 2022 (no desertó completamente)

In [28]:
max_dif_entre_mat_rut_carr = matriculados[
    (matriculados["mrun"].isin(ruts_matriculados_cft_ip))
    & (
        (matriculados["anio_titulacion"].notna())
        | (matriculados["ultimo_anio_matricula"] == 2022)
    )
].groupby(["mrun", "codigo_unico"]).agg({
    "cat_periodo": get_array_diff
}).reset_index().rename(columns={
    "cat_periodo": "anios_en_blanco"
})

max_dif_entre_mat_rut_carr.head()

,mrun,codigo_unico,anios_en_blanco
0,19.0,I536S0C39J1V1,1
1,98.0,I111S22C8J1V1,1
2,99.0,I111S12C420J1V1,1
3,118.0,I221S1C113J2V1,1
4,196.0,I280S8C380J1V1,1


In [42]:
tiene_blanco = max_dif_entre_mat_rut_carr[max_dif_entre_mat_rut_carr["anios_en_blanco"] > 1]
tiene_blanco_cant_carreras = matriculados[
    matriculados["mrun"].isin(tiene_blanco["mrun"])
].groupby(["mrun"]).agg({"codigo_unico": pd.Series.nunique}).reset_index()

tiene_blanco_n_carr = tiene_blanco_cant_carreras[tiene_blanco_cant_carreras["codigo_unico"] > 1]
matriculados[
    matriculados["mrun"].isin(tiene_blanco_n_carr["mrun"].unique())
].sort_values(by=["mrun", "codigo_unico", "cat_periodo"])[["mrun", "codigo_unico", "cat_periodo", "ultimo_anio_matricula", "anio_titulacion"]]

,mrun,codigo_unico,cat_periodo,ultimo_anio_matricula,anio_titulacion
293611,2299.0,I123S2C47J1V1,2016,2016,NaN
694367,2299.0,I123S2C56J1V1,2017,2020,2021.0
1174413,2299.0,I123S2C56J1V1,2018,2020,2021.0
2251344,2299.0,I123S2C56J1V1,2020,2020,2021.0
293621,3266.0,I106S3C43J2V1,2016,2016,NaN
...,...,...,...,...,...
2774539,26230830.0,I16S1C53J2V2,2020,2020,NaN
3322888,26230830.0,I430S14C42J2V3,2021,2021,NaN
3322907,26230889.0,I155S2C67J1V1,2021,2021,NaN
2774568,26230889.0,I155S2C67J2V1,2020,2022,NaN


In [ ]:
ruts_desertores_2_1 = max_dif_entre_mat_rut_carr[max_dif_entre_mat_rut_carr["anios_en_blanco"] == 2][["mrun", "codigo_unico"]].drop_duplicates()
ruts_desertores_2_2 = max_dif_entre_mat_rut_carr[max_dif_entre_mat_rut_carr["anios_en_blanco"] == 3][["mrun", "codigo_unico"]].drop_duplicates()
ruts_desertores_2_3 = max_dif_entre_mat_rut_carr[max_dif_entre_mat_rut_carr["anios_en_blanco"] == 4][["mrun", "codigo_unico"]].drop_duplicates()

print(f"Cant. matriculas de CFT/IP que debieron terminar carrera y congelaron 1 año: ", ruts_desertores_2_1.shape[0])
print(f"Cant. matriculas de CFT/IP que debieron terminar carrera y congelaron 2 años: ", ruts_desertores_2_2.shape[0])
print(f"Cant. matriculas de CFT/IP que debieron terminar carrera y congelaron 3 años: ", ruts_desertores_2_3.shape[0])

### Tercer caso: Vuelve a otra carrera en la misma institución
- No se han titulado de su primera carrera
- Puede o no tener "años en banco"
- Se titula de su enésima carrera o se ha matriculado en el año 2022 (no desertó completamente)